In [3]:
import os
os.getenv("GROQ_API_KEY")

'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'

In [6]:
from dotenv import load_dotenv
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [9]:
from langchain_groq import ChatGroq
llm_model= "deepseek-r1-distill-llama-70b"
model = ChatGroq(model=llm_model)
output = model.invoke("Hi, how are you")
print(output)

content="<think>\n\n</think>\n\nHi! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How are you doing? 😊" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 8, 'total_tokens': 50, 'completion_time': 0.192073741, 'prompt_time': 0.000154178, 'queue_time': 0.09490871299999999, 'total_time': 0.192227919}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'stop', 'logprobs': None} id='run--b6aaa76a-a686-4f2a-b670-9515d8d5b145-0' usage_metadata={'input_tokens': 8, 'output_tokens': 42, 'total_tokens': 50}


In [16]:
import operator 
from typing import List
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import StateGraph, MessagesState, START, END

In [ ]:
def call_model(state: MessagesState):
    message = state["messages"]